## Cài đặt các thư viện

In [1]:
!pip install transformers datasets torch evaluate tqdm rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5c32b1ee72bdba962551a57e1ab5c5dc1c7b5a24d12ce29f14a30af45e35833f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from rouge_score import rouge_scorer

## Chuẩn bị dữ liệu

In [3]:
from datasets import load_dataset

# Tải dữ liệu
dataset = load_dataset("nhantruongcse/summary-vietnamese-news")

# Số lượng bản ghi trong dữ liệu
total_size = 10000

# Chia tỷ lệ phần trăm
train_size = int(total_size * 0.8)  # 80% cho train
val_size = int(total_size * 0.1)    # 10% cho validation
test_size = total_size - train_size - val_size  # Còn lại cho test

# Lấy ra tập dữ liệu 'train' từ DatasetDict và chia tỷ lệ
train_dataset = dataset["train"].select(range(0, train_size))
val_dataset = dataset["train"].select(range(train_size, train_size + val_size))
test_dataset = dataset["train"].select(range(train_size + val_size, total_size))

# Kiểm tra kết quả
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")


README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/165M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/161M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/164573 [00:00<?, ? examples/s]

Train size: 8000
Validation size: 1000
Test size: 1000


In [4]:
# Hàm tiền xử lý dữ liệu
def preprocess_data(example):
    return {
        "input_text": "summarize: " + example["Content"],  # Thêm tiền tố 'summarize:'
        "summary_text": example["Summary"]
    }

# Tiền xử lý dữ liệu cho các tập train, validation và test
preprocessed_train = train_dataset.map(preprocess_data, batched=False)
preprocessed_val = val_dataset.map(preprocess_data, batched=False)
preprocessed_test = test_dataset.map(preprocess_data, batched=False)

# Lọc bỏ các cột không cần thiết, giữ lại chỉ 'input_text' và 'summary_text'
preprocessed_train = preprocessed_train.remove_columns([col for col in preprocessed_train.column_names if col not in ["input_text", "summary_text"]])
preprocessed_val = preprocessed_val.remove_columns([col for col in preprocessed_val.column_names if col not in ["input_text", "summary_text"]])
preprocessed_test = preprocessed_test.remove_columns([col for col in preprocessed_test.column_names if col not in ["input_text", "summary_text"]])

# Kiểm tra kết quả sau khi lọc
print(preprocessed_train[0])  # Hiển thị mẫu đầu tiên sau khi lọc cột


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'input_text': 'summarize: Hội nghị về giải pháp và định hướng phát triển nghiên cứu ứng dụng tại Việt Nam đã được Bộ Y tế tổ chức hôm nay 6.12, tại Hà Nội. Tại hội nghị, TS Nguyễn Ngô Quang, Phó cục trưởng Cục Khoa học công nghệ và đào tạo (Bộ Y tế), cho biết tại Việt Nam, điều trị tế bào, đặc biệt là tế bào gốc còn mang tính tự phát, nhiều cơ sở y tế ứng dụng không đúng quy định. Đặc biệt, nhiều quảng cáo không được kiểm chứng khoa học, có thể gây hậu quả cho người hoặc hiệu quả không rõ ràng gây tốn kém. TS Nguyễn Ngô Quang khẳng định, ứng dụng trị liệu tế bào và sản phẩm từ tế bào được quản lý chặt chẽ mà vẫn khuyến khích được nghiên cứu TRẦN MINH "Nhiều người tại Việt Nam đã sang các quốc gia lân cận chi số tiền lớn cho trị liệu tế bào gốc. Chúng tôi cũng nghe nhiều người hỏi "các quốc gia khác đều điều trị tế bào gốc vô tư, thoải mái nhưng tại sao "quản" chặt chẽ thế?", ông Quang chia sẻ. Trả lời câu hỏi trên, Phó cục trưởng Cục Khoa học công nghệ và đào tạo cho biết: "Chúng tôi 

In [5]:
# Kiểm tra kết quả
print(preprocessed_train[0])  # Hiển thị mẫu đầu tiên


{'input_text': 'summarize: Hội nghị về giải pháp và định hướng phát triển nghiên cứu ứng dụng tại Việt Nam đã được Bộ Y tế tổ chức hôm nay 6.12, tại Hà Nội. Tại hội nghị, TS Nguyễn Ngô Quang, Phó cục trưởng Cục Khoa học công nghệ và đào tạo (Bộ Y tế), cho biết tại Việt Nam, điều trị tế bào, đặc biệt là tế bào gốc còn mang tính tự phát, nhiều cơ sở y tế ứng dụng không đúng quy định. Đặc biệt, nhiều quảng cáo không được kiểm chứng khoa học, có thể gây hậu quả cho người hoặc hiệu quả không rõ ràng gây tốn kém. TS Nguyễn Ngô Quang khẳng định, ứng dụng trị liệu tế bào và sản phẩm từ tế bào được quản lý chặt chẽ mà vẫn khuyến khích được nghiên cứu TRẦN MINH "Nhiều người tại Việt Nam đã sang các quốc gia lân cận chi số tiền lớn cho trị liệu tế bào gốc. Chúng tôi cũng nghe nhiều người hỏi "các quốc gia khác đều điều trị tế bào gốc vô tư, thoải mái nhưng tại sao "quản" chặt chẽ thế?", ông Quang chia sẻ. Trả lời câu hỏi trên, Phó cục trưởng Cục Khoa học công nghệ và đào tạo cho biết: "Chúng tôi 

## Tải mô hình và tokenize dữ liệu

In [6]:
# Tải tokenizer và model
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
model = AutoModelForSeq2SeqLM.from_pretrained("vinai/bartpho-syllable")

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [7]:
# Hàm tokenize dữ liệu
def tokenize_function(example):
    # Tokenize đầu vào (input_text) và nhãn (summary_text)
    input_encodings = tokenizer(example['input_text'], max_length=512, truncation=True, padding="max_length")
    target_encodings = tokenizer(example['summary_text'], max_length=128, truncation=True, padding="max_length")
    
    # Thêm labels vào dữ liệu
    input_encodings["labels"] = target_encodings["input_ids"]
    
    # Trả về dữ liệu với các trường cần thiết
    return {**input_encodings, "labels": input_encodings["labels"]}

# Tokenize các tập dữ liệu
tokenized_train = preprocessed_train.map(tokenize_function, batched=True, batch_size=128)
tokenized_val = preprocessed_val.map(tokenize_function, batched=True, batch_size=128)
tokenized_test = preprocessed_test.map(tokenize_function, batched=True, batch_size=128)

# Kiểm tra một mẫu dữ liệu trong tập train đã được tokenize
print(tokenized_train[0])  # Hiển thị mẫu đầu tiên trong tập train đã được tokenize


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'input_text': 'summarize: Hội nghị về giải pháp và định hướng phát triển nghiên cứu ứng dụng tại Việt Nam đã được Bộ Y tế tổ chức hôm nay 6.12, tại Hà Nội. Tại hội nghị, TS Nguyễn Ngô Quang, Phó cục trưởng Cục Khoa học công nghệ và đào tạo (Bộ Y tế), cho biết tại Việt Nam, điều trị tế bào, đặc biệt là tế bào gốc còn mang tính tự phát, nhiều cơ sở y tế ứng dụng không đúng quy định. Đặc biệt, nhiều quảng cáo không được kiểm chứng khoa học, có thể gây hậu quả cho người hoặc hiệu quả không rõ ràng gây tốn kém. TS Nguyễn Ngô Quang khẳng định, ứng dụng trị liệu tế bào và sản phẩm từ tế bào được quản lý chặt chẽ mà vẫn khuyến khích được nghiên cứu TRẦN MINH "Nhiều người tại Việt Nam đã sang các quốc gia lân cận chi số tiền lớn cho trị liệu tế bào gốc. Chúng tôi cũng nghe nhiều người hỏi "các quốc gia khác đều điều trị tế bào gốc vô tư, thoải mái nhưng tại sao "quản" chặt chẽ thế?", ông Quang chia sẻ. Trả lời câu hỏi trên, Phó cục trưởng Cục Khoa học công nghệ và đào tạo cho biết: "Chúng tôi 

In [8]:
# Hàm collate_fn để xử lý batch
def collate_fn(batch):
    # Kiểm tra batch để lọc các mẫu hợp lệ
    filtered_batch = [item for item in batch if "input_ids" in item and "attention_mask" in item and "labels" in item]
    
    # Kiểm tra nếu filtered_batch không rỗng
    if len(filtered_batch) == 0:
        # Trả về batch rỗng nếu không có mẫu hợp lệ
        return {"input_ids": torch.tensor([]), "attention_mask": torch.tensor([]), "labels": torch.tensor([])}

    # Nếu batch không rỗng, stack tensor
    input_ids = torch.stack([torch.tensor(item["input_ids"]) for item in filtered_batch])
    attention_mask = torch.stack([torch.tensor(item["attention_mask"]) for item in filtered_batch])
    labels = torch.stack([torch.tensor(item["labels"]) for item in filtered_batch])

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Giả sử tokenized_train, tokenized_val, tokenized_test là các danh sách đã được tiền xử lý trước
# DataLoader cho train, validation, và test
train_loader = DataLoader(tokenized_train, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(tokenized_val, batch_size=8, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(tokenized_test, batch_size=8, shuffle=False, collate_fn=collate_fn)

# Kiểm tra một batch
for batch in train_loader:
    print(batch['input_ids'])  # Kiểm tra input_ids
    print(batch['labels'])     # Kiểm tra labels
    break  # Chỉ in ra batch đầu tiên

tensor([[    0, 37054, 14961,  ...,     1,     1,     1],
        [    0, 37054, 14961,  ...,     1,     1,     1],
        [    0, 37054, 14961,  ...,   119,     3,     2],
        ...,
        [    0, 37054, 14961,  ...,     4,    10,     2],
        [    0, 37054, 14961,  ...,  3224,   654,     2],
        [    0, 37054, 14961,  ...,     1,     1,     1]])
tensor([[   0,  468, 1517,  ...,    1,    1,    1],
        [   0,  552,   28,  ...,    1,    1,    1],
        [   0,  190,  240,  ...,    1,    1,    1],
        ...,
        [   0,  403,  427,  ...,    1,    1,    1],
        [   0, 1625,  101,  ...,    1,    1,    1],
        [   0, 1132,    9,  ...,    1,    1,    1]])


## Huấn luyện và đánh giá

In [9]:
from rouge_score import rouge_scorer
import torch
from tqdm import tqdm

def train_model(model, train_loader, val_loader, optimizer, device, tokenizer, num_epochs=3, scheduler=None):
    model.to(device)  # Đảm bảo mô hình được chuyển sang thiết bị đúng (GPU hoặc CPU)
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Training phase
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc="Training"):
            # Bỏ qua các batch nếu collate_fn trả về None
            if batch is None:
                continue

            input_ids = batch["input_ids"].to(device)  # Chuyển sang device
            attention_mask = batch["attention_mask"].to(device)  # Chuyển sang device
            labels = batch["labels"].to(device)  # Chuyển sang device

            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            train_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Average training loss: {avg_train_loss:.4f}")

        # Scheduler step if exists (optional)
        if scheduler:
            scheduler.step()

        # Evaluation phase
        model.eval()
        val_loss = 0
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []

        rouge_scorer_obj = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Evaluating"):
                # Bỏ qua các batch nếu collate_fn trả về None
                if batch is None:
                    continue

                input_ids = batch["input_ids"].to(device)  # Chuyển sang device
                attention_mask = batch["attention_mask"].to(device)  # Chuyển sang device
                labels = batch["labels"].to(device)  # Chuyển sang device

                # Forward pass (prediction)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()  # Cộng dồn loss cho tập validation

                # ROUGE score calculation
                predictions = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4)
                pred_texts = [tokenizer.decode(g, skip_special_tokens=True) for g in predictions]
                ref_texts = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]

                # Tính ROUGE score cho từng cặp dự đoán và tham chiếu
                for pred, ref in zip(pred_texts, ref_texts):
                    scores = rouge_scorer_obj.score(pred, ref)
                    rouge1_scores.append(scores["rouge1"].fmeasure)
                    rouge2_scores.append(scores["rouge2"].fmeasure)
                    rougeL_scores.append(scores["rougeL"].fmeasure)

        avg_val_loss = val_loss / len(val_loader) if len(val_loader) > 0 else 0
        avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
        avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
        avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

        print(f"Average validation loss: {avg_val_loss:.4f}")
        print(f"ROUGE-1: {avg_rouge1:.4f}")
        print(f"ROUGE-2: {avg_rouge2:.4f}")
        print(f"ROUGE-L: {avg_rougeL:.4f}")
        print("=" * 50)


In [10]:
# Định nghĩa thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Định nghĩa optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Huấn luyện mô hình
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    tokenizer=tokenizer,
    num_epochs=6
)

Using device: cuda
Epoch 1/6


Training: 100%|██████████| 1000/1000 [17:10<00:00,  1.03s/it]


Average training loss: 0.9757


Evaluating: 100%|██████████| 125/125 [06:35<00:00,  3.17s/it]


Average validation loss: 0.6156
ROUGE-1: 0.5664
ROUGE-2: 0.2973
ROUGE-L: 0.3749
Epoch 2/6


...

Average validation loss: 0.7585
ROUGE-1: 0.5954
ROUGE-2: 0.2980
ROUGE-L: 0.3823
Epoch 6/6


Training: 100%|██████████| 1000/1000 [17:10<00:00,  1.03s/it]


Average training loss: 0.3114


Evaluating: 100%|██████████| 125/125 [05:08<00:00,  2.47s/it]

Average validation loss: 0.8117
ROUGE-1: 0.5971
ROUGE-2: 0.2998
ROUGE-L: 0.3856


In [11]:
def test_model(model, test_loader, tokenizer, device):
    """
    Evaluate the model on the test set and compute ROUGE scores.
    """
    model.eval()  # Đặt mô hình ở chế độ đánh giá
    rouge_scorer_obj = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    # Khởi tạo danh sách để lưu ROUGE scores
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

    with torch.no_grad():  # Tắt gradient để tiết kiệm bộ nhớ
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Dự đoán kết quả
            predictions = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4)
            pred_texts = [tokenizer.decode(g, skip_special_tokens=True) for g in predictions]
            ref_texts = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]

            # Tính ROUGE scores cho mỗi cặp dự đoán và tham chiếu
            for pred, ref in zip(pred_texts, ref_texts):
                scores = rouge_scorer_obj.score(pred, ref)
                rouge1_scores.append(scores["rouge1"].fmeasure)
                rouge2_scores.append(scores["rouge2"].fmeasure)
                rougeL_scores.append(scores["rougeL"].fmeasure)

    # Tính điểm trung bình của ROUGE scores
    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

    return {
        "ROUGE-1": avg_rouge1,
        "ROUGE-2": avg_rouge2,
        "ROUGE-L": avg_rougeL
    }

In [12]:
# Gọi hàm test_model để đánh giá mô hình
results = test_model(
    model=model,
    test_loader=test_loader,
    device=device,
    tokenizer=tokenizer
)

print("ROUGE scores on the test set:")
print(f"ROUGE-1: {results['ROUGE-1']:.4f}")
print(f"ROUGE-2: {results['ROUGE-2']:.4f}")
print(f"ROUGE-L: {results['ROUGE-L']:.4f}")

Testing: 100%|██████████| 125/125 [04:26<00:00,  2.13s/it]

ROUGE scores on the test set:
ROUGE-1: 0.5927
ROUGE-2: 0.2862
ROUGE-L: 0.3761


In [13]:
# Đường dẫn lưu trữ
save_path = "/kaggle/working/saved_model"

# Lưu mô hình và tokenizer sau khi fine-tune
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer have been saved at: {save_path}")



Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


Model and tokenizer have been saved at: /kaggle/working/saved_model


In [14]:
model_path = "/kaggle/working/saved_model"

# Tải mô hình và tokenizer đã lưu
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Văn bản đầu vào
input_text = """Tổ chức Hợp tác và phát triển kinh tế (OECD) vừa công bố năm 2022. Theo đó, học sinh Việt Nam đạt 469 điểm toán, 462 điểm đọc hiểu và 472 điểm khoa học, thấp hơn 3 - 14 điểm so với mức trung bình của các nước OECD. Kết quả và thứ hạng của Việt Nam tại PISA 2022, công bố ngày 5.12 OECD So với kỳ đánh giá năm 2018, điểm toán trung bình của học sinh Việt Nam giảm 27 điểm, đọc hiểu và khoa học giảm lần lượt 43 và 71 điểm. Xét về thứ hạng, học sinh Việt Nam nằm ở mức trung bình về môn toán, nhưng dưới trung bình ở môn đọc hiểu và khoa học. Cụ thể, trong số 73 nước và 8 vùng lãnh thổ tham gia PISA năm 2022 thì Việt Nam đứng thứ 31 về toán, 34 về đọc hiểu và 37 về khoa học. Từ khi Việt Nam tham gia xếp hạng PISA vào năm 2012, kết quả xếp hạng năm nay thấp nhất, giảm bậc ở tất cả các lĩnh vực. Trong đó, kết quả môn toán học giảm 7 - 14 bậc, đọc hiểu giảm 2 - 21 bậc, khoa học 27 - 31 bậc. Việt Nam đã có 4 lần tham gia PISA. Ở lần đầu tham gia, năm 2012, Việt Nam đạt kết quả khá với 511 điểm lĩnh vực toán, 508 điểm lĩnh vực đọc hiểu và 528 điểm lĩnh vực khoa học. Tới năm 2015, PISA tập trung vào lĩnh vực khoa học, cũng là lần thứ 2 Việt Nam tham gia và đạt kết quả ở mức trên điểm trung bình OECD (lĩnh vực khoa học đạt 525 điểm, toán học đạt 495 điểm và đọc hiểu đạt 487 điểm). Năm 2018, học sinh Việt Nam đạt 505 điểm đọc hiểu, cao thứ 13 trong 79 quốc gia và vùng lãnh thổ tham gia, tăng 19 bậc so với chu kỳ năm 2015. Về toán học, Việt Nam đạt 496 điểm, cao thứ 24. Còn với lĩnh vực khoa học, Việt Nam đạt 543 điểm, cao thứ 4. Theo đánh giá của báo cáo, giảm điểm là xu hướng chung của kết quả PISA 2022, riêng các nước trong khối OECD sụt giảm "chưa từng có". Học sinh các quốc gia OECD giảm trung bình 15 điểm toán, 11 điểm đọc hiểu và 2 điểm khoa học. Tại Hội thảo khoa học về phương pháp giáo dục và đánh giá phẩm chất năng lực người học do Viện Khoa học giáo dục Việt Nam tổ chức hôm nay 6.12, Viện trưởng, GS Lê Anh Vinh nêu nhận xét: một xu hướng trong cả 4 lần thi PISA của Việt Nam, đó là chúng ta có tỷ lệ nhóm điểm thấp nhất và tỷ lệ nhóm điểm cao nhất là tương đối thấp so với các nước có điểm tương đương. Cũng theo GS Lê Anh Vinh, nếu như phân tích vào các nhóm điểm thì chúng ta sẽ thấy trong nhóm 25% kết quả cao nhất và 25% thấp nhất của Việt Nam thì khoảng cách điểm là khoảng 78 điểm. Số điểm này tương đương với khoảng 2 năm rưỡi học tập. Đáng chú ý, khoảng cách này đang cao hơn khoảng cách của năm đầu tiên chúng ta tham gia PISA vào năm 2015 (năm đó khoảng cách là hơn 60 điểm). Tuy nhiên, khoảng cách này vẫn thấp so với mặt bằng chung của OECD là hơn 90 điểm (khoảng cách khoảng 3 năm học). GS Vinh nhấn mạnh: "Khoảng cách điểm này là một khoảng cách rất lớn giữa học sinh có điều kiện học tập tốt nhất và những học sinh khó khăn nhất. Các con có thể chênh nhau bằng khoảng 3 năm học tập tại trường và chắc chắn chúng ta phải làm rất nhiều điều để có thể thu hẹp khoảng cách này". GS Lê Anh Vinh đặt vấn đề: "Kỳ thi PISA năm 2022 dự kiến tổ chức vào năm 2021, tuy nhiên phải lùi lại 1 năm so với dự kiến . Chắc chắn dịch bệnh này ảnh hưởng tới chất lượng giáo dục trên toàn thế giới nhưng câu chuyện là "khó người khó ta", tại sao tất cả đều bị ảnh hưởng bởi đại dịch mà nước ta có vẻ lại bị ảnh hưởng nặng hơn so với các nước khác hay không?". Ông Vinh cũng cho rằng: "Kết quả PISA của Việt Nam vẫn rất tốt. Ở đây, tôi muốn đặt câu hỏi: liệu rằng học sinh Việt Nam trong 2 năm vừa qua do dịch bệnh, chúng ta duy trì được việc học nhưng việc kiểm tra, đánh giá thường xuyên, việc thi vào lớp 10 có gì thay đổi làm ảnh hưởng đến kết quả hay không? Điều này cần có tìm hiểu kỹ hơn, sâu hơn nhưng rõ ràng chúng ta nhìn thấy việc kiểm tra, đánh giá trong suốt quá trình học tập sẽ ảnh hưởng đến kết quả chung của học sinh". PISA (Programme for International Student Assessment) là chương trình đánh giá học sinh quốc tế, do Tổ chức Hợp tác và phát triển kinh tế (OECD) khởi xướng với quy mô toàn cầu, chu kỳ 3 năm một lần, đánh giá năng lực của học sinh 15 tuổi ở ba lĩnh vực là đọc hiểu, toán, khoa học. Mỗi kỳ sẽ có một lĩnh vực được lựa chọn để đánh giá sâu hơn, gọi là lĩnh vực trọng tâm và sử dụng làm căn cứ để xếp loại chất lượng giáo dục của các quốc gia. Năm 2018 là lĩnh vực đọc hiểu. Chu kỳ đầu tiên của PISA được đánh giá là năm 2000. Ban đầu, PISA được thiết kế đánh giá bằng đề thi trên giấy. Tuy nhiên, đến năm 2018, hầu hết quốc gia đã chuyển sang hình thức thi trên máy tính, chỉ còn 9 nước tham gia thi trên giấy, trong đó có Việt Nam."""
# Tokenize văn bản đầu vào
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

# Sinh tóm tắt
summary_ids = model.generate(
    inputs["input_ids"], 
    attention_mask=inputs["attention_mask"],  # Bổ sung attention mask
    max_length=150,       # Chiều dài tối đa
    min_length=30,        # Đảm bảo tóm tắt có độ dài tối thiểu
    num_beams=5,          # Số lượng beam search cao hơn để cải thiện chất lượng
    no_repeat_ngram_size=3,  # Ngăn cụm từ lặp lại
    length_penalty=1.5,   # Ưu tiên tóm tắt dài hơn
    early_stopping=True
)

# Giải mã tóm tắt
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Tóm tắt:", summary)

Tóm tắt: Kết quả xếp hạng của Việt Nam tại PISA 2022 theo đó tụt hạng so với lần đánh giá gần nhất năm 2018, khiến vị trí của nước này trong bảng xếp hạng theo sau Mỹ, Trung Quốc và các nước G7 mở rộng.


In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
model.push_to_hub("summaization_vietnamese")
tokenizer.push_to_hub("summaization_vietnamese")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ChiHieuNguyen/summaization_vietnamese/commit/36d814d32c74987273485f934862692b295d3a7c', commit_message='Upload tokenizer', commit_description='', oid='36d814d32c74987273485f934862692b295d3a7c', pr_url=None, pr_revision=None, pr_num=None)